In [1]:
import numpy as np
import re
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import cPickle as pickle
import re

In [2]:
dataset_location = '../Wikipedia/Dataset.p'
instances = pickle.load( open( dataset_location, "rb" ) )

## Run statistics on the corpus

In [3]:
excludedPredicates = ['http://www.w3.org/2000/01/rdf-schema#seeAlso', 'http://dbpedia.org/ontology/wikiPageRedirects']

num_triples = []
for key in instances:
    for j in range(0, len(instances[key]['annotated_sentence'])):
        for z in range(len(instances[key]['triples'][j]) -1, -1, -1):
            tempPredicate = instances[key]['triples'][j][z].split()[1]
            if tempPredicate in excludedPredicates:
                instances[key]['triples'][j].pop(z)
        if len(instances[key]['triples'][j]) >= 1 and len(instances[key]['triples'][j]) <= 15:
            num_triples.append(len(instances[key]['triples'][j]))
num_triples = np.asarray(num_triples)
print ('Mean: %.2f' % num_triples.mean())
print ('Standard Deviation: %.2f' % num_triples.std())

Mean: 2.53
Standard Deviation: 1.98


In [4]:
dataset_list = []
for key in instances:
    print key
    for j in range(0, len(instances[key]['annotated_sentence'])):
        if len(instances[key]['triples'][j]) >= 1:
            dataset_list.append({'annotated_sentence': instances[key]['annotated_sentence'][j], 'triples': instances[key]['triples'][j]})

Gennady_Padalka
Robert_L._Crippen
S._David_Griggs
Kalpana_Chawla
Linda_M._Godwin
Nancy_J._Currie
James_McDivitt
John_E._Blaha
Aleksandr_Aleksandrovich_Skvortsov
Roger_B._Chaffee
Vitaly_Sevastyanov
Charles_Bassett
Steve_MacLean_(astronaut)
Oleg_Kononenko
Klaus_von_Storch
Eberhard_KФllner
Zhai_Zhigang
Frank_Borman
Wang_Yaping
Fred_Haise
Valeri_Tokarev
Sigmund_JДhn
Hans_Schlegel
Yury_Lonchakov
Roberta_Bondar
Sergei_Zalyotin
William_J._Knight
Tracy_Caldwell_Dyson
Robert_Crippen
Edgar_Mitchell
Hero_of_the_USSR
Enos_the_Chimp
Valery_Korzun
Christopher_Ferguson
Joseph_R._Tanner
Roy_D._Bridges,_Jr
Kenneth_Ham
Samuel_T._Durrance
John_B._Herrington
Robert_Thirsk
Joe_F._Edwards,_Jr
Rusty_Schweickart
Franco_Malerba
John_M._Grunsfeld
Karl_Gordon_Henize
Michel_Tognini
William_Frederick_Fisher
Cosmonaut
C._Gordon_Fullerton
Stephen_Thorne_(astronaut)
Gennadi_Sarafanov
David_A._Wolf
Rakesh_Sharma
Yelena_Serova
Yelena_Kondakova
Leonid_Kadeniuk
Mike_Melvill
Albert_Sacco
Pamela_A._Melroy
Richard_N._Richar

In [5]:
print (len(dataset_list))

4499


## Finding distribution of predicates across sentences.

In [6]:
pred2sentence = {}
for i in range(len(dataset_list) - 1, -1, -1):
    for j in range(0, len(dataset_list[i]['triples'])):
        tempTriple = dataset_list[i]['triples'][j].split()
        tempPredicate = tempTriple[1]
        if tempPredicate not in pred2sentence:
            pred2sentence[tempPredicate] = [i]
        else:
            pred2sentence[tempPredicate].append(i)
            

In [7]:
sentence_id = []

for pred in pred2sentence:
    if len(pred2sentence[pred]) >= 100:
        print ('%s: %d' % (pred, len((pred2sentence[pred]))))
        for id in pred2sentence[pred]:
            if id not in sentence_id:
                sentence_id.append(id)
sentence_id.sort()

http://dbpedia.org/property/crew1Up: 338
http://dbpedia.org/ontology/type: 1054
http://dbpedia.org/property/parentAgency: 143
http://dbpedia.org/ontology/isPartOf: 166
http://dbpedia.org/ontology/nationality: 520
http://dbpedia.org/property/crew4Up: 134
http://dbpedia.org/property/crewMembers: 1319
http://dbpedia.org/ontology/country: 204
http://dbpedia.org/ontology/birthPlace: 655
http://dbpedia.org/property/crew3Up: 217
http://www.w3.org/1999/02/22-rdf-syntax-ns#type: 1323
http://dbpedia.org/ontology/mission: 1618
http://dbpedia.org/property/launchSite: 167
http://dbpedia.org/property/crew5Up: 116
http://dbpedia.org/property/rank: 286
http://dbpedia.org/property/crew2Up: 308
http://dbpedia.org/ontology/occupation: 274


## Create corpus according to the sentence that have been dictate by the predicates distribution.

In [8]:
processed_list = []
for sent in sentence_id:
    processed_list.append(dataset_list[sent])

In [9]:
len(processed_list)

3831

In [10]:
def clean_annotations(string):
    formal = True
    annotations = re.findall(r'(\[\[[0-9a-zA-Z_.,\-\(\)\%\'\|\ ]+\]\])', string)
    if len(annotations) > 0:
        for annotation in annotations:
            if formal:
                replacement = re.findall(r'\|([0-9a-zA-Z_.,\-\(\)\%\']+)\]\]', annotation)
            else:
                replacement = re.findall(r'\[\[([0-9a-zA-Z_.,\-\(\)\%\'\ ]+)\|', annotation)
            string = string.replace(annotation, replacement[0])
        clean_annotations(string)
    return string

In [11]:
namespaces = ['(http\:\/\/dbpedia\.org/\w+\/)', 'http\:\/\/www\.w3\.org\/1999\/02\/22\-rdf\-syntax\-ns\#', 'http\:\/\/www\.w3\.org\/2000\/01\/rdf\-schema\#']
def clean_namespace(triples):
    for j in range(0, len(triples)):
        for namespace in namespaces:
            triples[j] = re.sub(re.compile(namespace, re.DOTALL), '', triples[j])\
              .replace('\t', '').replace('\n', '') 
    return triples

In [12]:
def highlight(triples):
    for j in range(0, len(triples)):
        triples[j] = triples[j].split()
        triples[j][1] = '<b><font color="red">' + triples[j][1] + '</font></b>'
        triples[j] = ' '.join(triples[j])
    return triples

In [13]:
def clean_annotations(original, triples, unlinked_entities):
    #print('Original')
    #print original, triples
    annotations = re.findall(r'(\[\[[0-9a-zA-Z_.,\-\(\)\%\'\|\ ]+\]\])', original)
    if len(annotations) > 0:
        for annotation in annotations:
            formal_part = re.findall(r'\|([0-9a-zA-Z_.,\-\(\)\%\']+)\]\]', annotation)[0]
            informal_part = re.findall(r'\[\[([0-9a-zA-Z_.,\-\(\)\%\'\ ]+)\|', annotation)[0]
            original = original.replace(annotation, informal_part)
            #print simplification
            #print formal_part
            for i in range(0, len(triples)):
                triples[i] = triples[i].split()
                if formal_part == triples[i][0]:
                    triples[i][0] = informal_part
                    if formal_part in unlinked_entities:
                        unlinked_entities.remove(formal_part)
                if formal_part == triples[i][-1]:
                    triples[i][-1] = informal_part
                    if formal_part in unlinked_entities:
                        unlinked_entities.remove(formal_part)
                triples[i] = ' '.join(triples[i])
                """
                if formal_part in triples[i]:
                    #namespace = re.findall(r'([0-9a-zA-Z_.,\-\(\)\%\'\:\/])+{}'.format(re.escape(formal_part)), triples[i])[0]
                    namespace = re.findall(r'({})'.format(re.escape(formal_part)), triples[i])[0]
                    triples[i] = triples[i].replace(namespace, informal_part)
                """
        clean_annotations(original, triples, unlinked_entities)
    return original, triples, unlinked_entities

### The `included_triples` list is important for calculating the final distribution of the predicates in the resultant corpus.

In [14]:
included_triples = []

def exclude(triples, unlinked_entities):
    for entity in unlinked_entities:
        for i in range(len(triples) - 1, -1, -1):
            triples[i] = triples[i].split()
            if triples[i][0] == entity or triples[i][-1] == entity:
                triples[i] = ' '.join(triples[i])
                print('Removing triple-fact: %s ...' % (triples[i]))
                triples.pop(i)
            else: triples[i] = ' '.join(triples[i])
    return triples

def setup_experiment(list):
    #num_sentences = len(list)
    incl_sentences = []
    #while len(incl_sentences) < num_sentences:
    for sentence in range(len(list) - 1, -1, -1):
        #sentence = np.random.randint(0, len(list))
        entities = []
        init_triples = highlight(clean_namespace(list[sentence]['triples']))
        for i in range(0, len(init_triples)):
                init_triples[i] = init_triples[i].split()
                if init_triples[i][0] not in entities: entities.append(init_triples[i][0])
                if init_triples[i][-1] not in entities: entities.append(init_triples[i][-1])
                init_triples[i] = ' '.join(init_triples[i])
        annotated_sentence, triples, unlinked_entities = clean_annotations(list[sentence]['annotated_sentence'], \
                                              init_triples, entities)
        triples = exclude(triples, unlinked_entities)
        if len(triples) > 0:
            included_triples.append(triples)
            incl_sentences.append({'Original Sentence': annotated_sentence})
            incl_sentences[len(incl_sentences) - 1]['Triple-Fact 1'] = triples[0]         
            for j in range(1, len(list[sentence]['triples'])):
                incl_sentences[len(incl_sentences) - 1][('Triple-Fact %d' % (j + 1))] = triples[j]
        #list.pop(sentence)
    return incl_sentences    

In [15]:
#output_df = pd.DataFrame(setup_experiment(wiki_list + medline_list))
output_df = pd.DataFrame(setup_experiment(processed_list))
output_df.to_csv('../Data/Experiment.csv', index=False)

Removing triple-fact: Soyuz TM-23 <b><font color="red">crewMembers</font></b> Yuri_Onufrienko ...
Removing triple-fact: Soyuz TM-23 <b><font color="red">crewMembers</font></b> Yuri_Onufrienko ...
Removing triple-fact: EO-21 <b><font color="red">crewMembers</font></b> Yuri_Onufrienko ...
Removing triple-fact: Mir-21 <b><font color="red">crewMembers</font></b> Yuri_Onufrienko ...
Removing triple-fact: Soyuz TM-23 <b><font color="red">crewMembers</font></b> Yuri_Onufrienko ...
Removing triple-fact: Mir EO-21 <b><font color="red">crewMembers</font></b> Yuri_Onufrienko ...
Removing triple-fact: Michael_T._Good <b><font color="red">type</font></b> astronauts ...
Removing triple-fact: Michael_T._Good <b><font color="red">type</font></b> NASA ...
Removing triple-fact: Michael_T._Good <b><font color="red">birthPlace</font></b> Parma,_Ohio ...
Removing triple-fact: Broadview Heights <b><font color="red">northwest</font></b> Parma,_Ohio ...
Removing triple-fact: Parma,_Ohio <b><font color="red">s

## Finding distribution of predicates across the resultant dataset.

In [16]:
pred2sentence = {}
for i in range(0, len(included_triples)):
    for j in range(0, len(included_triples[i])):
        tempTriple = included_triples[i][j]
        tempPredicate = re.findall(r"(?<=color\=\"red\"\>)(.*)(?=<\/font>)", tempTriple)[0]
        if tempPredicate not in pred2sentence:
            pred2sentence[tempPredicate] = [i]
        else:
            pred2sentence[tempPredicate].append(i)

In [17]:
for pred in pred2sentence:
    if len(pred2sentence[pred]) >= 80:
        print('%s: %d' % (pred, len(pred2sentence[pred])))

mission: 520
occupation: 117
programme: 84
crewMembers: 388
crew2Up: 104
isPartOf: 159
nationality: 325
country: 204
rank: 190
launchSite: 181
crew1Up: 121
type: 1444
parentAgency: 141
birthPlace: 374


In [19]:
pred2sentence.keys()

[u'activitySector',
 u'jurisdiction',
 u'demonym',
 u'mission',
 u'federalSubject',
 u'battle',
 u'occupation',
 u'agency3Up',
 u'northeast',
 u'title',
 u'lastStopover',
 u'departureCraft',
 u'knownFor',
 u'instrument',
 u'locationCountry',
 u'parents',
 u'location',
 u'agency7Up',
 u'aux',
 u'east',
 u'crew8Up',
 u'crew3Up',
 u'relatedOccupation',
 u'predecessor',
 u'variantsWithTheirOwnArticles',
 u'launchPad',
 u'routeEnd',
 u'headquarter',
 u'seats1Title',
 u'landingSite',
 u'awardedBy',
 u'successor',
 u'artist',
 u'officialLang',
 u'presenter',
 u'rockets',
 u'expedition5Down',
 u'chief1Name',
 u'athletics',
 u'programme',
 u'payloadItems',
 u'populationPlace',
 u'superseding',
 u'regionServed',
 u'house',
 u'crewMembers',
 u'related',
 u'seat',
 u'owner',
 u'crew7Up',
 u'crew2Up',
 u'shortDescription',
 u'event',
 u'establishedEvent',
 u'parent',
 u'admCtrName',
 u'agency2Up',
 u'derivedfrom',
 u'routeJunction',
 u'agency6Up',
 u'hostCity',
 u'religion',
 u'state',
 u'moreUsers